<a href="https://colab.research.google.com/github/MedAljane/Stage_Project_files/blob/main/coronas_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deep-translator

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from deep_translator import GoogleTranslator
import re
import pandas as pd
import os
import urllib.request
import time

In [ ]:
#Liste des pays
countrieslinks = []
aTags = BeautifulSoup(requests.get("https://www.worldometers.info/coronavirus/#countries").content, 'html.parser').find('table', {'id': 'main_table_countries_today'}).find('tbody').findAll('a', {'class': 'mt_a'})
for a in aTags:
  countrieslinks.append(f"Insert ==> {a['href'][a['href'].index('/')+1:-1]} <== for: {a.text}")


paysTarget = input('Enter country name: ')
try:
  soup = BeautifulSoup(requests.get('https://www.worldometers.info/coronavirus/country/'+paysTarget).content , 'html.parser')
  dates = []
  cases = []
  r = soup.find('div', {'class': 'news_date'})
  if r != None:
    dates.append(re.sub(r'\s+', ' ', re.sub(r'\[[0-9]*\]', ' ', r.text.replace(' (GMT)', ''))))

    for i in soup.findAll('button',{'class':'btn btn-light date-btn'}):
        dates.append(re.sub(r'\[[0-9]*\]', ' ', re.sub(r'\s+', ' ', i.text)))
    cases=[]
    cases.append(re.sub(r'\s+', ' ', re.sub(r'\[[0-9]*\]', ' ', r.find_next_sibling('div').findChild('li', {'class': 'news_li'}, recursive=True).text)))
    for i in soup.findAll('div',{'class':'newsdate_div'}):
        cases.append(re.sub(r'\s+', ' ', re.sub(r'\[[0-9]*\]', ' ', i.text)))
    cases = [e[:e.index('in')-1].replace('Updates ','') for e in cases]
    cases = [GoogleTranslator(source='en', target='fr').translate(e) for e in cases]
    dates = [GoogleTranslator(source='en', target='fr').translate(e) for e in dates]
    corona = [[i,j] for i,j in zip(dates,cases)]
  else:
    raise Exception("Country not in list!")
except Exception as ex:
  print(ex, '\nTo know what to put check ==> countrieslinks <== variable!')
coronaDict = {
    "date": dates,
    "nouveaux_cas": [e.split('et')[0].strip() for e in cases],
    "nouveaux_deces": [e.split('et')[1].strip() for e in cases]
}

Enter country name: Tunisia


In [ ]:
df = pd.DataFrame(coronaDict)
df

,date,nouveaux_cas,nouveaux_deces
0,7 août,2 909 nouveaux cas,130 nouveaux décès
1,6 août,2 448 nouveaux cas,129 nouveaux décès
2,5 août,3 163 nouveaux cas,140 nouveaux décès
3,4 août,2 819 nouveaux cas,184 nouveaux décès
4,3 août,1 243 nouveaux cas,159 nouveaux décès
5,2 août,2 651 nouveaux cas,209 nouveaux décès
6,1 août,3 316 nouveaux cas,172 nouveaux décès


In [ ]:
df.to_json('coronaCases.json', orient='records', lines=True, force_ascii=False)